# GPT

1. Selenium으로 지정된 로그인 페이지에 접속해 ID와 비밀번호를 자동 입력하고 로그인 버튼을 클릭한다.  
2. 로그인 완료 후 브라우저의 쿠키에서 `SESSION_KEY` 값만 찾아 추출한다.  
3. 추출한 `SESSION_KEY` 값을 콘솔에 출력한다.  


In [3]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

LOGIN_PAGE_URL = "http://210.96.71.85/auth/login/loginView.do"
LOGIN_DATA = {
    "user_id": "jeonje",
    "password": "qwasqwas12"
}

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)

try:
    driver.get(LOGIN_PAGE_URL)
    wait = WebDriverWait(driver, 10)

    # 로그인 입력
    wait.until(EC.presence_of_element_located((By.NAME, "user_id"))).send_keys(LOGIN_DATA["user_id"])
    wait.until(EC.presence_of_element_located((By.NAME, "password"))).send_keys(LOGIN_DATA["password"])
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn-login"))).click()

    # main.do 로드 대기
    wait.until(EC.url_contains("main.do"))

    # SESSION_KEY만 추출
    cookies = driver.get_cookies()
    session_key = next((c["value"] for c in cookies if c["name"] == "SESSION_KEY"), None)

finally:
    driver.quit()

if session_key:
    print(f"session_key: {session_key}")  # 오직 SESSION_KEY 값만 출력
else:
    print("SESSION_KEY 없음")


session_key: 935595238145748416


- 알고리즘 최적화

In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

LOGIN_PAGE_URL = "http://210.96.71.85/auth/login/loginView.do"
LOGIN_DATA = {
    "user_id": "jeonje",
    "password": "qwasqwas12",
}

# ---- Chrome 옵션 (속도/로그 최적화) ----
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--no-sandbox")                      # Windows에서도 무해, 컨텍스트 단순화
options.add_argument("--disable-dev-shm-usage")           # 공유 메모리 이슈 완화
options.add_argument("--disable-extensions")
options.add_argument("--disable-background-networking")
options.add_argument("--disable-default-apps")
options.add_argument("--disable-notifications")
options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로딩 차단
options.add_experimental_option('excludeSwitches', ['enable-logging'])  # 콘솔 로그 억제
# 페이지 로드 전략: DOMContentLoaded까지만 대기
options.set_capability("pageLoadStrategy", "eager")

# Chromedriver 자체 로그 버리기(속도엔 큰 영향 없지만 콘솔 깔끔)
service = Service(log_path=os.devnull)

driver = webdriver.Chrome(options=options, service=service)
driver.set_page_load_timeout(15)  # 무한 대기 방지

try:
    driver.get(LOGIN_PAGE_URL)

    # 대기 설정: 짧고 자주 체크
    wait = WebDriverWait(driver, timeout=6, poll_frequency=0.2)

    # 입력 필드 로드 후 즉시 입력
    user_el = wait.until(EC.presence_of_element_located((By.NAME, "user_id")))
    pwd_el  = wait.until(EC.presence_of_element_located((By.NAME, "password")))
    user_el.clear(); user_el.send_keys(LOGIN_DATA["user_id"])
    pwd_el.clear();  pwd_el.send_keys(LOGIN_DATA["password"])

    # 버튼 클릭: 렌더 지연 회피용 JS 클릭(기본 클릭보다 빠른 경우 있음)
    btn = driver.find_element(By.CLASS_NAME, "btn-login")
    driver.execute_script("arguments[0].click();", btn)

    # 페이지 전환을 길게 기다리지 않고, 세션 쿠키가 생기는지만 체크
    # (일반적으로 인증 성공 시 쿠키가 즉시 설정됨)
    def has_session_cookie(drv):
        ck = drv.get_cookie("SESSION_KEY")
        return ck["value"] if ck and ck.get("value") else False

    session_key = WebDriverWait(driver, timeout=5, poll_frequency=0.2).until(has_session_cookie)

finally:
    driver.quit()

if session_key:
    print(f"session_key: {session_key}")
else:
    print("SESSION_KEY 없음")


session_key: 935595239617448417


# 스튜디오

1. Selenium으로 로그인 페이지를 열고 ID(`user_id`)와 비밀번호를 자동 입력한 뒤 로그인 버튼을 클릭한다.  
2. 로그인 완료 시점까지 대기 후 브라우저 쿠키에서 `SESSION_KEY` 값을 찾아 저장한다.  
3. `SESSION_KEY`가 있으면 출력하고, 없으면 실패 메시지를 표시한 뒤 브라우저를 종료한다.  


In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. URL 및 로그인 정보
login_page_url = "http://210.96.71.85/auth/login/loginView.do"

# HTML의 name 속성에 맞춰 키(key) 이름을 'user_id'로 변경 (가독성 향상)
login_data = {
    'user_id': 'jeonje',
    'password': 'qwasqwas12'
}

# 웹드라이버 옵션 설정
options = webdriver.ChromeOptions()
# --- 디버깅 시에는 아래 줄을 주석 처리하여 눈으로 확인하세요 ---
options.add_argument("--headless")
# --------------------------------------------------------
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36")

driver = None
try:
    # 2. 크롬 드라이버 실행
    driver = webdriver.Chrome(options=options)
    print("브라우저를 실행합니다...")
    
    # 3. 로그인 페이지로 이동
    driver.get(login_page_url)
    print(f"'{driver.title}' 페이지로 이동했습니다.")

    wait = WebDriverWait(driver, 10)

    # 4. ID와 비밀번호 입력 (★★★★★ 수정된 부분)
    # name 속성이 'user_id'인 요소를 찾아 로그인 ID를 입력
    print("ID 입력란을 찾습니다...")
    id_input = wait.until(EC.presence_of_element_located((By.NAME, "user_id")))
    id_input.send_keys(login_data['user_id'])
    print("ID를 입력했습니다.")

    # name 속성이 'password'인 요소를 찾아 비밀번호를 입력
    # (HTML상 password 필드의 name이 'password'일 것으로 추정, 오류 시 이 부분 확인)
    password_input = driver.find_element(By.NAME, "password")
    password_input.send_keys(login_data['password'])
    print("비밀번호를 입력했습니다.")

    # 5. 로그인 버튼 클릭 (★★★★★ 수정된 부분)
    # class 이름이 'btn-login'인 요소를 찾아 클릭
    print("로그인 버튼을 찾습니다...")
    login_button = driver.find_element(By.CLASS_NAME, 'btn-login')
    login_button.click()
    print("로그인 버튼을 클릭했습니다.")

    # 6. 로그인이 완료되고 페이지가 전환될 때까지 대기
    print("로그인 완료를 기다립니다...")
    # 로그인 후 URL이 'main.do'를 포함하는지 확인하여 성공 여부 판단
    wait.until(EC.url_contains("main.do"))
    print("로그인 성공! 메인 페이지로 이동했습니다.")

    # 7. 세션 쿠키 추출
    all_cookies = driver.get_cookies()
    
    print("\n--- 획득한 쿠키 정보 ---")
    session_key = None
    for cookie in all_cookies:
        print(f"  {cookie['name']} = {cookie['value']}")
        if cookie['name'] == 'SESSION_KEY':
            session_key = cookie['value']
    print("--------------------------\n")

    if session_key:
        print(f"✅ 성공: 최종 SESSION_KEY 값을 가져왔습니다 -> {session_key}")
    else:
        print("❌ 실패: 로그인에 성공했으나 쿠키에서 SESSION_KEY를 찾을 수 없습니다.")

finally:
    # 8. 작업이 끝나면 브라우저를 종료
    if driver:
        # time.sleep(3) # 작동 확인을 위해 잠시 대기
        driver.quit()
        print("브라우저를 종료했습니다.")

브라우저를 실행합니다...
'DestinyECM:V' 페이지로 이동했습니다.
ID 입력란을 찾습니다...
ID를 입력했습니다.
비밀번호를 입력했습니다.
로그인 버튼을 찾습니다...
로그인 버튼을 클릭했습니다.
로그인 완료를 기다립니다...
로그인 성공! 메인 페이지로 이동했습니다.

--- 획득한 쿠키 정보 ---
  edmframe-left-size = 181
  destiny.system.cookie.LastUsersLocale = ko
  lastNetworkMode = 0
  CLIENT_LOCALE = ko
  SESSION_KEY = 935595235146248415
  JSESSIONID = A1EE06BCDF7AE39410A4624D3E7E8626
--------------------------

✅ 성공: 최종 SESSION_KEY 값을 가져왔습니다 -> 935595235146248415
브라우저를 종료했습니다.
